In [1]:
import pandas as pd

callrate = pd.read_csv('./call_rate.csv')

# 날짜를 기준으로 정렬하기
callrate = callrate.sort_values(by=['date'])
# callrate.head(10)

In [2]:
def all_date_range_df(date_range):
    df2 = pd.DataFrame({'date':date_range})
    
    return df2

# 비어있는 날짜와 금리를 추가하기
date_range = (pd.date_range(start='20041224', end='20200831')).strftime("%Y.%m.%d").tolist()
df2 = all_date_range_df(date_range)

In [3]:
def df_outer_join_merge(df1, df2):
    
    # 두 df을 '날짜'키 값을 기준으로 outer_join 병합
    df_OUTER_JOIN = pd.merge(df1, df2, left_on='date', right_on='date', how='outer', sort=True)
    
    # 두 값중 결측값을 앞 방향으로 채우기 (공휴일에 없는 환율정보는 전날을 기준으로 결측값 채우기)
    df_finally = df_OUTER_JOIN.fillna(method='ffill')
    
    return df_finally

fin_df = df_outer_join_merge(callrate, df2)
fin_df = fin_df.set_index('date')
# fin_df.head(40)

In [4]:
# 한달 전의 날짜 추가하기
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

month_date = []
for target_date in fin_df.index:
    
    month_before = (datetime.strptime(target_date,'%Y.%m.%d' ) - relativedelta(months = 1)).strftime('%Y.%m.%d')
    month_date.append(month_before)
    
fin_df['month_date'] = month_date

In [5]:
# 한달 전의 날짜에 맞춰 콜 금리 추가하기
month_rate=[]
for target_date in fin_df['month_date']:
    try:
        rate = fin_df._get_value(target_date, 'callrate') 
        month_rate.append(rate)
    except: # keyError 제외
        month_rate.append(0.0)
fin_df['month_rate'] = month_rate
fin_df

,callrate,month_date,month_rate
date,,,
2004.12.24,3.29,2004.11.24,0.00
2004.12.25,3.29,2004.11.25,0.00
2004.12.26,3.29,2004.11.26,0.00
2004.12.27,3.29,2004.11.27,0.00
2004.12.28,3.27,2004.11.28,0.00
...,...,...,...
2020.08.27,0.50,2020.07.27,0.49
2020.08.28,0.49,2020.07.28,0.47
2020.08.29,0.49,2020.07.29,0.48


In [11]:
# 콜 금리의 차이를 구하고 0 초과 : 'up', 0 같음 : 'same', 0 미만 : 'down'으로 라벨링
fin_df['value_diff'] = fin_df['callrate'] - fin_df['month_rate']

fin_df['label']='0'
for i, diff in enumerate(fin_df['value_diff']):
    print(diff)
    if diff > 0:
        fin_df['label'][i] = 'up'
    elif diff == 0:
        fin_df['label'][i] = 'same'
    elif diff < 0:
        fin_df['label'][i] = 'down'   
fin_df

3.29
3.29
3.29
3.29
3.27
3.26
3.28
3.29
3.29
3.29
3.27
3.27
3.28
3.21
3.22
3.22
3.22
3.27
3.28
3.27
3.28
3.26
3.26
3.26
3.25
3.25
3.25
3.23
3.19
3.19
3.19
-0.040000000000000036
-0.040000000000000036
-0.03000000000000025
-0.04999999999999982
-0.020000000000000018
-0.009999999999999787
-0.029999999999999805
-0.03000000000000025
-0.03000000000000025
-0.040000000000000036
-0.010000000000000231
-0.04999999999999982
-0.05999999999999961
0.010000000000000231
-0.040000000000000036
-0.040000000000000036
-0.040000000000000036
-0.08999999999999986
-0.009999999999999787
0.0
-0.009999999999999787
0.0
0.0
0.0
0.020000000000000018
0.009999999999999787
0.009999999999999787
0.029999999999999805
0.06999999999999984
0.03000000000000025
0.08999999999999986
0.020000000000000018
0.029999999999999805
0.020000000000000018
0.03999999999999959
0.029999999999999805
0.020000000000000018
0.029999999999999805
0.010000000000000231
0.029999999999999805
0.029999999999999805
0.029999999999999805
-0.010000000000000231
0

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.



0.009999999999999787
0.0
0.0
0.009999999999999787
0.0
0.0
0.009999999999999787
-0.009999999999999787
-0.009999999999999787
-0.009999999999999787
0.040000000000000036
0.029999999999999805
-0.020000000000000018
-0.04999999999999982
0.23999999999999977
0.23999999999999977
0.23999999999999977
0.23999999999999977
0.25
0.25
0.2599999999999998
0.25
0.25
0.25
0.25
0.25
0.2599999999999998
0.26000000000000023
0.2200000000000002
0.25
0.2400000000000002
0.2400000000000002
0.25
0.2400000000000002
0.25
0.2599999999999998
0.2599999999999998
0.2599999999999998
0.2599999999999998
0.25
0.25
0.25
0.2200000000000002
0.2599999999999998
0.21999999999999975
-0.040000000000000036
0.010000000000000231
0.010000000000000231
0.010000000000000231
0.0
-0.010000000000000231
-0.020000000000000018
-0.010000000000000231
0.0
0.0
0.0
-0.040000000000000036
-0.029999999999999805
0.020000000000000018
0.040000000000000036
0.0
0.0
0.009999999999999787
0.009999999999999787
0.009999999999999787
0.009999999999999787
0.0
-0.0099

0.11000000000000032
0.03000000000000025
0.0600000000000005
0.08000000000000007
0.08000000000000007
0.08000000000000007
0.08999999999999986
0.08000000000000007
0.05999999999999961
-0.1899999999999995
-0.20000000000000018
-0.22000000000000064
-0.20000000000000018
0.020000000000000462
0.020000000000000462
0.020000000000000462
0.009999999999999787
0.019999999999999574
-0.009999999999999787
-0.009999999999999787
-0.009999999999999787
-0.009999999999999787
-0.009999999999999787
-0.040000000000000036
0.09999999999999964
0.20999999999999996
0.2599999999999998
0.04999999999999982
0.08000000000000007
0.15000000000000036
0.4299999999999997
0.47999999999999954
0.5099999999999998
0.47999999999999954
0.25
0.25
0.13999999999999968
0.1200000000000001
0.09000000000000075
0.29000000000000004
0.34000000000000075
0.3700000000000001
0.28000000000000025
0.019999999999999574
0.019999999999999574
0.0
-0.009999999999999787
-0.019999999999999574
-0.009999999999999787
-0.019999999999999574
-0.019999999999999574


-1.0899999999999994
-1.15
-1.0100000000000002
-1.0000000000000004
-1.0000000000000004
-1.0000000000000004
-1.0000000000000004
-1.0100000000000002
-1.0100000000000002
-1.0100000000000002
-1.06
-1.04
-1.04
-0.9799999999999995
-0.9099999999999997
-0.9099999999999997
-1.0000000000000004
-0.9900000000000002
-1.0000000000000004
-1.0000000000000004
-0.30000000000000027
-0.25
-0.25
-0.21999999999999975
-0.16999999999999948
-0.1599999999999997
-0.11999999999999966
-0.10999999999999943
-0.18999999999999995
-0.36999999999999966
-0.1200000000000001
0.7399999999999998
-0.3600000000000003
-0.3700000000000001
-0.8199999999999998
-1.08
-1.08
-1.08
-1.15
-1.2000000000000002
-1.12
-1.1400000000000001
-1.1300000000000003
-1.1700000000000004
-1.1600000000000001
-1.2799999999999998
-1.4100000000000001
-1.33
-1.3199999999999998
-0.9899999999999998
-0.9899999999999998
-1.0
-1.02
-1.0500000000000003
-0.9700000000000002
-0.9500000000000002
-1.1099999999999999
-1.1099999999999999
-1.04
-1.1
-1.2600000000000002


...

0.0
0.0
0.0
0.0
0.0
0.010000000000000009
0.010000000000000009
0.010000000000000009
0.0
0.010000000000000009
0.0
0.0
0.0
0.019999999999999796
0.039999999999999813
0.039999999999999813
0.029999999999999805
0.029999999999999805
0.029999999999999805
0.020000000000000018
0.0
0.010000000000000009
0.0
-0.029999999999999805
-0.010000000000000009
0.0
0.010000000000000009
0.010000000000000009
0.010000000000000009
-0.010000000000000009
-0.010000000000000009
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
-0.009999999999999787
-0.029999999999999805
-0.029999999999999805
-0.019999999999999796
-0.019999999999999796
-0.019999999999999796
-0.020000000000000018
0.010000000000000009
0.0
0.0
0.0
0.020000000000000018
0.010000000000000009
-0.010000000000000009
-0.010000000000000009
-0.010000000000000009
-0.010000000000000009
0.010000000000000009
0.0
0.0
0.0
0.0
0.0
-0.010000000000000009
-0.010000000000000009
-0.010000000000000009
-0.010000000000000009
-0.010000000000000009
0.0
0.0
-0.010000000000000009

-0.06000000000000005
-0.08000000000000007
-0.040000000000000036
-0.010000000000000009
0.0
-0.010000000000000009
-0.040000000000000036
-0.06000000000000005
-0.010000000000000009
-0.010000000000000009
-0.010000000000000009
-0.010000000000000009
-0.010000000000000009
-0.030000000000000027
-0.030000000000000027
-0.030000000000000027
-0.010000000000000009
0.0
0.0
0.0
-0.010000000000000009
-0.010000000000000009
-0.010000000000000009
0.0
0.0
0.0
0.0
0.020000000000000018
0.020000000000000018
0.020000000000000018
0.030000000000000027
0.030000000000000027
0.050000000000000044
0.040000000000000036
0.030000000000000027
0.020000000000000018
0.020000000000000018
0.050000000000000044
0.07000000000000006
0.020000000000000018
0.020000000000000018
0.020000000000000018
0.030000000000000027
0.030000000000000027
0.050000000000000044
0.040000000000000036
0.040000000000000036
0.030000000000000027
0.020000000000000018
0.010000000000000009
0.020000000000000018
0.020000000000000018
0.020000000000000018
0.020000

0.24
0.22999999999999998
0.24
0.25
0.28
0.25
0.26
0.25
0.25
0.26
0.33000000000000007
0.3400000000000001
0.33000000000000007
0.06000000000000005
0.1299999999999999
0.1299999999999999
0.07000000000000006
0.030000000000000027
0.07000000000000006
0.020000000000000018
0.040000000000000036
-0.010000000000000009
-0.010000000000000009
-0.020000000000000018
-0.010000000000000009
-0.010000000000000009
-0.020000000000000018
0.0
-0.030000000000000027
-0.020000000000000018
-0.010000000000000009
-0.010000000000000009
0.0
0.0
-0.020000000000000018
-0.020000000000000018
-0.010000000000000009
0.0
-0.020000000000000018
-0.010000000000000009
-0.020000000000000018
-0.06000000000000005
-0.06000000000000005
-0.08000000000000007
-0.08000000000000007
-0.07999999999999985
-0.1499999999999999
-0.09000000000000008
-0.030000000000000027
-0.010000000000000009
-0.010000000000000009
-0.010000000000000009
0.010000000000000009
0.010000000000000009
0.020000000000000018
0.0
-0.010000000000000009
0.0
-0.01000000000000000

0.030000000000000027
0.06
0.06
0.04999999999999999
0.06
0.07
0.08000000000000002
0.07
0.050000000000000044
0.040000000000000036
0.050000000000000044
0.010000000000000009
0.020000000000000018
0.010000000000000009
0.0
0.040000000000000036


,callrate,month_date,month_rate,value_diff,label
date,,,,,
2004.12.24,3.29,2004.11.24,0.00,3.29,up
2004.12.25,3.29,2004.11.25,0.00,3.29,up
2004.12.26,3.29,2004.11.26,0.00,3.29,up
2004.12.27,3.29,2004.11.27,0.00,3.29,up
2004.12.28,3.27,2004.11.28,0.00,3.27,up
...,...,...,...,...,...
2020.08.27,0.50,2020.07.27,0.49,0.01,up
2020.08.28,0.49,2020.07.28,0.47,0.02,up
2020.08.29,0.49,2020.07.29,0.48,0.01,up


In [ ]:
fin_df.reset_index(inplace=True)
fin_df

In [12]:
fin_df.loc[fin_df['label']=='down']

,callrate,month_date,month_rate,value_diff,label
date,,,,,
2005.01.24,3.25,2004.12.24,3.29,-0.04,down
2005.01.25,3.25,2004.12.25,3.29,-0.04,down
2005.01.26,3.26,2004.12.26,3.29,-0.03,down
2005.01.27,3.24,2004.12.27,3.29,-0.05,down
2005.01.28,3.25,2004.12.28,3.27,-0.02,down
...,...,...,...,...,...
2020.07.31,0.51,2020.06.30,0.57,-0.06,down
2020.08.05,0.48,2020.07.05,0.50,-0.02,down
2020.08.10,0.47,2020.07.10,0.49,-0.02,down


In [7]:
value_diff


date
2004.12.24    3.29
2004.12.25    3.29
2004.12.26    3.29
2004.12.27    3.29
2004.12.28    3.27
              ... 
2020.08.27    0.01
2020.08.28    0.02
2020.08.29    0.01
2020.08.30    0.00
2020.08.31    0.04
Length: 5730, dtype: float64